In [1]:
%run ../utils/common.py

In [2]:
processed_communities=get_processed_communities()

In [3]:
community_categories={}
with open("../data/categories_splitted_into_communities.json", "r") as f_in:
    for line in f_in:
        json_line=json.loads(line)
        community_id=json_line['community_id']
        if community_id not in processed_communities:
            continue
        categories=json_line['categories']
        community_categories[community_id]=categories

In [6]:
article_sections=load_json("../data/article_sections_filtered.json", object_hook=article_sections_object_hook)

In [7]:
category_articles={}
with open("../data/gini_articles_scores_0985_no_unknown_type.json") as f:
    for line in f:
        line_dict = json.loads(line)
        category = line_dict['category']
        articles=[article for article in line_dict['articles'] if article in article_sections.keys()]
        category_articles[category]=articles

In [8]:
for community_id in tqdm(processed_communities):
    id_section_map={}
    with open(f"../data/semantic_similarity/community_{community_id}/sentence_counter_by_section.json", "r") as f_in:
        for line in f_in:
            json_line=json.loads(line)
            
            id_section_map[json_line['id']]=json_line['section']
            
    section_id_map={v:k for k,v in id_section_map.items()}
            
    categories=community_categories[community_id]
    section_articles={}
    for category in categories:
        for article in category_articles[category]:
            for section in article_sections[article]:
                if section not in section_id_map.keys():
                    continue
                if section not in section_articles.keys():
                    section_articles[section]=set()
                section_articles[section].add(article)
    
    with open(f"../data/semantic_similarity/community_{community_id}/section_articles.json", "a+") as f_out:
        for section,articles in section_articles.items():
            f_out.write(json.dumps({'section':section,'articles':list(articles)})+"\n")
        

  0%|          | 0/7917 [00:00<?, ?it/s]